Recalculate the precipitation indices for seasonal effects just to see what difference that makes.

Also calculate annual indices for summer maxima - TX90, SU, TXX

In [2]:
# Import packages
import numpy as np
import pandas as pd
import xarray as xr
import datetime
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
from geocat.viz import util as gvutil 



In [3]:
import climdex.precipitation as pdex
indices = pdex.indices(time_dim='time')

iDir = "~/PROGS/Belmont FWE/LOCA-data/loca5/"
oDir = "~/PROGS/Belmont FWE/Indices/"

Define the precipitation functions

In [3]:
def yearly_rx1day(ds, varname = 'PRECT'):
    """
    Compute the annual maximum 24h precipitation total
    Args:
        ds: Dataset
        varname: assumes 'PRECT' 
    """
    ds = xr.DataArray(ds[varname].groupby('time.year').max('time'), name='RX1D')

    return ds
    
def yearly_rx5day(ds, varname = 'PRECT'):
    """
    Compute the annual maximum 5 day precipitation total
    Args:
        ds: Dataset
        varname: assumes 'PRECT'
    """
    ds = xr.DataArray(ds[varname].rolling(time=5).sum().groupby('time.year').max('time'), name='RX5D')

    return ds



In [98]:
def warmest_day(ds):
    """
    Compute maximum value of daily maximum temperature.
    Args:
        ds: Dataset.
    """
    ds = ds.groupby("time.year").max("time").to_array().squeeze().drop('variable')
    ds.name = 'TXX'
    ds.attrs["unit"], ds.attrs["longname"] = "deg C", "Maximum Annual Daily Maximum Temperature"  
    return ds


def summer_days(ds):
    """
    Compute days when Tx >25C
    Args:
        ds: Dataset.
     """
    ds = xr.where(ds['tasmax']>25,1,0).groupby('time.year').sum('time')
    ds.name = 'SU'
    ds.attrs["unit"], ds.attrs["longname"] = "days", "Summer Days; Tx greater than 25C"
    return ds




In [99]:
def t_quant(ds, threshold=0.9, time0='1981-01-01', time1='2010-12-31', varname='tasmax'):
    """
    Compute quantiles in temperature data. 
    Args:
        ds: Dataset.
        varname: TREFHTMX (assumes CESM output)
        threshold (float): Lower quantile percent (as decimal). Defaults to 0.9
        time0 (str): First time for slice. Defaults to 2015-01-01.
        time1 (str): Second time for slice. Defaults to 2034-12-31.
    """
    dp = ds.sel(time=slice(time0,time1)).quantile(q=[threshold],dim=['time']).squeeze().drop('quantile')
    dp = dp.to_array(name=(f"Q{str(int(float(threshold)*100))}"))
    return dp


def annualnum_above_q(ds, thresh_data, threshold=0.9, varname='tasmax'):
    """
    Compute number of days exceeding threshold per year
    Args:
        ds: Dataset
        thresh_data: Threshold dataset (computed using t_quant over all ensemble members)
        threshold: upper quantile percent as decimal, defaults to 0.9
        varname: TREFHTMX assumes CESM output
    """
    vee = 'TX'
    ds = xr.where(ds[varname] > thresh_data,1,0).groupby('time.year').sum('time')
    ds = ds.to_array(name = (f"{vee}{str(int(float(threshold)*100))}")).squeeze().drop('variable')
    return ds

In [4]:
### Create some standard information to include in each of the datasets for ease of writing
years = np.arange(1980,2100)
winyears = np.arange(1981,2099)
seasons = ('DJF', 'MAM', 'JJA', 'SON')
proj= ['cesm1-cam5.1.rcp45', 'cesm1-cam5.1.rcp85']
lat = np.linspace(39.53125,42.34375,num=46)
#lon = np.linspace(-76.21875,-74.40625, num=30)
lon = np.linspace(283.78125,285.59375, num=30) # calculate with original longs and then convert later
dims = ("proj", 'year', 'lat', 'lon')
attribs = dict(description='Seasonal Precipitation Indices based on ETCCDI definitions for LOCA data over Delaware River Basin. Using CESM1/RCP4.5 and RCP8.5.', 
                history='Created by Mari Tye December 2022.' )

In [5]:
# OPen precipitation data
with xr.open_dataset(iDir + 'Extraction_pr.nc') as dp:
    dp

# extra step to reassign december to next year

dp_win = dp.isel(time=(dp.time.dt.season=='DJF'))
dp_win['win_year'] = (dp_win.time.dt.month >= 12) + dp_win.time.dt.year
dp_win.coords['win_year'] = dp_win['win_year']
dp_spr = dp.isel(time=(dp.time.dt.season=='MAM'))
dp_sum = dp.isel(time=(dp.time.dt.season=='JJA'))
dp_aut = dp.isel(time=(dp.time.dt.season=='SON'))

In [209]:
# OPen temperature data
with xr.open_dataset(iDir + 'Extraction_tasmax.nc') as dtx:
    dtx

# extra step to reassign december to next year

dtx_win = dtx.isel(time=(dtx.time.dt.season=='DJF'))
dtx_win['win_year'] = (dtx_win.time.dt.month >= 12) + dtx_win.time.dt.year
dtx_win.coords['win_year'] = dtx_win['win_year']
dtx_spr = dtx.isel(time=(dtx.time.dt.season=='MAM'))
dtx_sum = dtx.isel(time=(dtx.time.dt.season=='JJA'))
dtx_aut = dtx.isel(time=(dtx.time.dt.season=='SON'))

Seasonal values of precipitation

In [252]:
# Create some blank arrays to fill with each index for each projection. Write these out at the end of the loop
Rx1d = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='RX1D')
Rx5d = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='RX5D')
PTOT = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='PRCPTOT')
sdii = xr.DataArray(None, coords=dict(proj=proj,  year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='SDII')
r10 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R10mm')
r20 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R20mm')
CWD = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='CWD')
CDD = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='CDD')

for r in range(2):
    print(proj[r])
    
    ds = dp_win.sel(projection=r)
    #### Calculate indices
    Rx1d[r,:,:,:] = xr.DataArray(ds['pr'].groupby('win_year').max(), name='RX1D')[:120,:,:]
    print('rx1d')
    Rx5d[r,:,:,:] = xr.DataArray(ds['pr'].rolling(time=5).sum().groupby('win_year').max(), name='RX5D')[:120,:,:]
    print('rx5d')
    PTOT[r,:,:,:] = xr.DataArray(ds['pr'].groupby('win_year').sum(), name='PRCPTOT')[:120,:,:]
    print('ptot')
    sdii[r,:,:,:] = xr.DataArray(indices.sdii(ds, period='1M', varname='pr').rolling(time=3).mean().groupby('time.year').mean(), name='SDII')
    print('sdii')

    ## Fixed threshold count of days
    r10[r,:,:,:] = xr.DataArray(indices.annual_r10mm(ds, varname='pr'), name='R10mm')
    print('r10')
    r20[r,:,:,:] = xr.DataArray(indices.annual_r20mm(ds, varname='pr'), name='R20mm')
    print('r20')

    ## Spells
    CWD[r,:,:,:] = xr.DataArray(indices.cwd(ds, period='1M', varname='pr').rolling(time=3).mean().groupby('time.year').mean(), name='CWD')
    print('cwd')
    CDD[r,:,:,:] = xr.DataArray(indices.cdd(ds, period='1M', varname='pr').rolling(time=3).mean().groupby('time.year').mean(),  name='CDD')
    print('cdd')

 #####
    rx1nm = 'LOCA_CESM1.RX1D.1981-2099.DJF.nc'
    rx5nm = 'LOCA_CESM1.RX5D.1981-2099.DJF.nc'
    ptotnm = 'LOCA_CESM1.PRCPTOT.1981-2099.DJF.nc'
    sdnm = 'LOCA_CESM1.SDII.1981-2099.DJF.nc'
    r10nm = 'LOCA_CESM1.R10mm.1981-2099.DJF.nc'
    r20nm = 'LOCA_CESM1.R20mm.1981-2099.DJF.nc'
    cddnm = 'LOCA_CESM1.CDD.1981-2099.DJF.nc'
    cwdnm = 'LOCA_CESM1.CWD.1981-2099.DJF.nc'
   
        # and write out to netcdf
    Rx1d.to_netcdf(rx1nm)
    Rx5d.to_netcdf(rx5nm)
    PTOT.to_netcdf(ptotnm)
    sdii.to_netcdf(sdnm)
    r10.to_netcdf(r10nm)
    r20.to_netcdf(r20nm)
    CWD.to_netcdf(cwdnm)
    CDD.to_netcdf(cddnm)



cesm1-cam5.1.rcp45
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
cesm1-cam5.1.rcp85
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd


In [270]:
# Create some blank arrays to fill with each index for each projection. Write these out at the end of the loop
Rx1d = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='RX1D')
Rx5d = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='RX5D')
PTOT = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='PRCPTOT')
sdii = xr.DataArray(None, coords=dict(proj=proj,  year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='SDII')
r10 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R10mm')
r20 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R20mm')
CWD = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='CWD')
CDD = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='CDD')

for r in range(2):
    print(proj[r])
    
    ds = dp_spr.sel(projection=r)
    #### Calculate indices
    Rx1d[r,:,:,:] = yearly_rx1day(ds, varname='pr')
    print('rx1d')
    Rx5d[r,:,:,:] = yearly_rx5day(ds, varname = 'pr')
    print('rx5d')
    PTOT[r,:,:,:] = xr.DataArray(indices.prcptot(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='PRCPTOT') # annual total
    print('ptot')
    sdii[r,:,:,:] = xr.DataArray(indices.sdii(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='SDII')
    print('sdii')

    ## Fixed threshold count of days
    r10[r,:,:,:] = xr.DataArray(indices.annual_r10mm(ds, varname='pr'), name='R10mm')
    print('r10')
    r20[r,:,:,:] = xr.DataArray(indices.annual_r20mm(ds, varname='pr'), name='R20mm')
    print('r20')

    ## Spells
    CWD[r,:,:,:] = xr.DataArray(indices.cwd(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='CWD')
    print('cwd')
    CDD[r,:,:,:] = xr.DataArray(indices.cdd(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='CDD')
    print('cdd')

 #####
rx1nm = 'LOCA_CESM1.RX1D.1981-2099.MAM.nc'
rx5nm = 'LOCA_CESM1.RX5D.1981-2099.MAM.nc'
ptotnm = 'LOCA_CESM1.PRCPTOT.1981-2099.MAM.nc'
sdnm = 'LOCA_CESM1.SDII.1981-2099.MAM.nc'
r10nm = 'LOCA_CESM1.R10mm.1981-2099.MAM.nc'
r20nm = 'LOCA_CESM1.R20mm.1981-2099.MAM.nc'
cddnm = 'LOCA_CESM1.CDD.1981-2099.MAM.nc'
cwdnm = 'LOCA_CESM1.CWD.1981-2099.MAM.nc'
   
        # and write out to netcdf
Rx1d.to_netcdf(rx1nm)
Rx5d.to_netcdf(rx5nm)
PTOT.to_netcdf(ptotnm)
sdii.to_netcdf(sdnm)
r10.to_netcdf(r10nm)
r20.to_netcdf(r20nm)
CWD.to_netcdf(cwdnm)
CDD.to_netcdf(cddnm)



cesm1-cam5.1.rcp45
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
cesm1-cam5.1.rcp85
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd


In [271]:
# Create some blank arrays to fill with each index for each projection. Write these out at the end of the loop
Rx1d = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='RX1D')
Rx5d = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='RX5D')
PTOT = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='PRCPTOT')
sdii = xr.DataArray(None, coords=dict(proj=proj,  year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='SDII')
r10 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R10mm')
r20 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R20mm')
CWD = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='CWD')
CDD = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='CDD')

for r in range(2):
    print(proj[r])
    
    ds = dp_sum.sel(projection=r)
    #### Calculate indices
    Rx1d[r,:,:,:] = yearly_rx1day(ds, varname='pr')
    print('rx1d')
    Rx5d[r,:,:,:] = yearly_rx5day(ds, varname = 'pr')
    print('rx5d')
    PTOT[r,:,:,:] = xr.DataArray(indices.prcptot(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='PRCPTOT') # annual total
    print('ptot')
    sdii[r,:,:,:] = xr.DataArray(indices.sdii(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='SDII')
    print('sdii')

    ## Fixed threshold count of days
    r10[r,:,:,:] = xr.DataArray(indices.annual_r10mm(ds, varname='pr'), name='R10mm')
    print('r10')
    r20[r,:,:,:] = xr.DataArray(indices.annual_r20mm(ds, varname='pr'), name='R20mm')
    print('r20')

    ## Spells
    CWD[r,:,:,:] = xr.DataArray(indices.cwd(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='CWD')
    print('cwd')
    CDD[r,:,:,:] = xr.DataArray(indices.cdd(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='CDD')
    print('cdd')

 #####
rx1nm = oDir + 'LOCA_CESM1.RX1D.1981-2099.JJA.nc'
rx5nm = oDir + 'LOCA_CESM1.RX5D.1981-2099.JJA.nc'
ptotnm = oDir + 'LOCA_CESM1.PRCPTOT.1981-2099.JJA.nc'
sdnm = oDir + 'LOCA_CESM1.SDII.1981-2099.JJA.nc'
r10nm = oDir + 'LOCA_CESM1.R10mm.1981-2099.JJA.nc'
r20nm = oDir + 'LOCA_CESM1.R20mm.1981-2099.JJA.nc'
cddnm = oDir + 'LOCA_CESM1.CDD.1981-2099.JJA.nc'
cwdnm = oDir + 'LOCA_CESM1.CWD.1981-2099.JJA.nc'
   
        # and write out to netcdf
Rx1d.to_netcdf(rx1nm)
Rx5d.to_netcdf(rx5nm)
PTOT.to_netcdf(ptotnm)
sdii.to_netcdf(sdnm)
r10.to_netcdf(r10nm)
r20.to_netcdf(r20nm)
CWD.to_netcdf(cwdnm)
CDD.to_netcdf(cddnm)



cesm1-cam5.1.rcp45
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
cesm1-cam5.1.rcp85
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd


In [7]:
# Just check the number of wet days
r1 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R1mm')

for r in range(2):
    print(proj[r])
    
    ds = dp_win.sel(projection=r)
    r1[r,:,:,:] = xr.DataArray(xr.where(ds['pr']>=1,1,0).groupby('win_year').sum(), name='R1mm')[:120,:,:]
    print('r1')

 #####
r1nm = oDir + 'LOCA_CESM1.R1mm.1981-2099.DJF.nc'
        # and write out to netcdf
r1.to_netcdf(r1nm)

cesm1-cam5.1.rcp45
r1
cesm1-cam5.1.rcp85
r1


In [272]:
# Create some blank arrays to fill with each index for each projection. Write these out at the end of the loop
Rx1d = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='RX1D')
Rx5d = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='RX5D')
PTOT = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='PRCPTOT')
sdii = xr.DataArray(None, coords=dict(proj=proj,  year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='SDII')
r10 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R10mm')
r20 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='R20mm')
CWD = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='CWD')
CDD = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='CDD')

for r in range(2):
    print(proj[r])
    
    ds = dp_aut.sel(projection=r)
    #### Calculate indices
    Rx1d[r,:,:,:] = yearly_rx1day(ds, varname='pr')
    print('rx1d')
    Rx5d[r,:,:,:] = yearly_rx5day(ds, varname = 'pr')
    print('rx5d')
    PTOT[r,:,:,:] = xr.DataArray(indices.prcptot(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='PRCPTOT') # annual total
    print('ptot')
    sdii[r,:,:,:] = xr.DataArray(indices.sdii(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='SDII')
    print('sdii')

    ## Fixed threshold count of days
    r10[r,:,:,:] = xr.DataArray(indices.annual_r10mm(ds, varname='pr'), name='R10mm')
    print('r10')
    r20[r,:,:,:] = xr.DataArray(indices.annual_r20mm(ds, varname='pr'), name='R20mm')
    print('r20')

    ## Spells
    CWD[r,:,:,:] = xr.DataArray(indices.cwd(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='CWD')
    print('cwd')
    CDD[r,:,:,:] = xr.DataArray(indices.cdd(ds, period='1Y', varname='pr').groupby('time.year').mean(), name='CDD')
    print('cdd')

 #####
rx1nm = oDir + 'LOCA_CESM1.RX1D.1981-2099.SON.nc'
rx5nm = oDir + 'LOCA_CESM1.RX5D.1981-2099.SON.nc'
ptotnm = oDir + 'LOCA_CESM1.PRCPTOT.1981-2099.SON.nc'
sdnm = oDir + 'LOCA_CESM1.SDII.1981-2099.SON.nc'
r10nm = oDir + 'LOCA_CESM1.R10mm.1981-2099.SON.nc'
r20nm = oDir + 'LOCA_CESM1.R20mm.1981-2099.SON.nc'
cddnm = oDir + 'LOCA_CESM1.CDD.1981-2099.SON.nc'
cwdnm = oDir + 'LOCA_CESM1.CWD.1981-2099.SON.nc'
   
        # and write out to netcdf
Rx1d.to_netcdf(rx1nm)
Rx5d.to_netcdf(rx5nm)
PTOT.to_netcdf(ptotnm)
sdii.to_netcdf(sdnm)
r10.to_netcdf(r10nm)
r20.to_netcdf(r20nm)
CWD.to_netcdf(cwdnm)
CDD.to_netcdf(cddnm)



cesm1-cam5.1.rcp45
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
cesm1-cam5.1.rcp85
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd


Seasonal and annual Temperature indices

In [186]:
# Create some blank arrays to fill with each index for each projection. Write these out at the end of the loop
TXX = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='TXX')
SU = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='SU')
Q90 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='Q90')

for r in range(2):
    print(proj[r])
    
    ds = dtx.sel(projection=r)
    #### Calculate indices
    TXX[r,:,:,:] = warmest_day(ds)
    print('txx')
    SU[r,:,:,:] = summer_days(ds)
    print('su')
    Q90[r,:,:,:] = t_quant(ds)
    print('q90')
    

 #####
    txxnm = oDir + 'LOCA_CESM1.TXX.1981-2099.nc'
    sunm = oDir + 'LOCA_CESM1.SU.1981-2099.nc'
    q90nm = oDir + 'LOCA_CESM1.Q90.1981-2099.nc'
    
    # and write out to netcdf
    TXX.to_netcdf(txxnm)
    SU.to_netcdf(sunm)
    Q90.to_netcdf(q90nm)
    


cesm1-cam5.1.rcp45
txx
su


/glade/u/home/maritye/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


q90
cesm1-cam5.1.rcp85
txx
su


/glade/u/home/maritye/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


q90


Only looking at high temps. So SU and TX90 in spring, summer and autumn.

In [202]:
# Create some blank arrays to fill with each index for each projection. Write these out at the end of the loop
SU = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='SU')
TX90 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='TX90')

for r in range(2):
    print(proj[r])
    
    ds = dtx_spr.sel(projection=r)
    dq = Q90.sel(proj=proj[r]).sel(year=slice('1981-01-01', '2010-12-31')).mean('year')
    #### Calculate indices
    SU[r,:,:,:] = summer_days(ds)
    print('su')
    TX90[r,:,:,:] = xr.DataArray(xr.where(ds['tasmax'] > dq,1,0).groupby('time.year').sum('time'), name='TX90')
    print('tx90')
    

 #####
    sunm = oDir + 'LOCA_CESM1.SU.1981-2099.MAM.nc'
    tx90nm = oDir + 'LOCA_CESM1.TX90.1981-2099.MAM.nc'
    
    # and write out to netcdf
    SU.to_netcdf(sunm)
    TX90.to_netcdf(tx90nm)
    


cesm1-cam5.1.rcp45


/glade/u/home/maritye/.local/lib/python3.7/site-packages/xarray/core/nanops.py:133: RuntimeWarning: invalid value encountered in true_divide
  data = data / (valid_count - ddof)


su
tx90
cesm1-cam5.1.rcp85


/glade/u/home/maritye/.local/lib/python3.7/site-packages/xarray/core/nanops.py:133: RuntimeWarning: invalid value encountered in true_divide
  data = data / (valid_count - ddof)


su
tx90


In [204]:
# Create some blank arrays to fill with each index for each projection. Write these out at the end of the loop
SU = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='SU')
TX90 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='TX90')

for r in range(2):
    print(proj[r])
    
    ds = dtx_sum.sel(projection=r)
    dq = Q90.sel(proj=proj[r]).sel(year=slice('1981-01-01', '2010-12-31')).mean('year')
    #### Calculate indices
    SU[r,:,:,:] = summer_days(ds)
    print('su')
    TX90[r,:,:,:] = xr.DataArray(xr.where(ds['tasmax'] > dq,1,0).groupby('time.year').sum('time'), name='TX90')
    print('tx90')
    

 #####
    sunm = oDir + 'LOCA_CESM1.SU.1981-2099.JJA.nc'
    tx90nm = oDir + 'LOCA_CESM1.TX90.1981-2099.JJA.nc'
    
    # and write out to netcdf
    SU.to_netcdf(sunm)
    TX90.to_netcdf(tx90nm)
    


cesm1-cam5.1.rcp45


/glade/u/home/maritye/.local/lib/python3.7/site-packages/xarray/core/nanops.py:133: RuntimeWarning: invalid value encountered in true_divide
  data = data / (valid_count - ddof)


su
tx90
cesm1-cam5.1.rcp85


/glade/u/home/maritye/.local/lib/python3.7/site-packages/xarray/core/nanops.py:133: RuntimeWarning: invalid value encountered in true_divide
  data = data / (valid_count - ddof)


su
tx90


In [203]:
# Create some blank arrays to fill with each index for each projection. Write these out at the end of the loop
SU = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='SU')
TX90 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='TX90')

for r in range(2):
    print(proj[r])
    
    ds = dtx_aut.sel(projection=r)
    dq = Q90.sel(proj=proj[r]).sel(year=slice('1981-01-01', '2010-12-31')).mean('year')
    #### Calculate indices
    SU[r,:,:,:] = summer_days(ds)
    print('su')
    TX90[r,:,:,:] = xr.DataArray(xr.where(ds['tasmax'] > dq,1,0).groupby('time.year').sum('time'), name='TX90')
    print('tx90')
    

 #####
    sunm = oDir + 'LOCA_CESM1.SU.1981-2099.SON.nc'
    tx90nm = oDir + 'LOCA_CESM1.TX90.1981-2099.SON.nc'
    
    # and write out to netcdf
    SU.to_netcdf(sunm)
    TX90.to_netcdf(tx90nm)
    


cesm1-cam5.1.rcp45


/glade/u/home/maritye/.local/lib/python3.7/site-packages/xarray/core/nanops.py:133: RuntimeWarning: invalid value encountered in true_divide
  data = data / (valid_count - ddof)


su
tx90
cesm1-cam5.1.rcp85


/glade/u/home/maritye/.local/lib/python3.7/site-packages/xarray/core/nanops.py:133: RuntimeWarning: invalid value encountered in true_divide
  data = data / (valid_count - ddof)


su
tx90
